#### In this notebook I start first from building a quick EDA revealing patters in the data, then I use EDA insights to build a simple linear model with only essential features (based on a out of time approach).

##### Since we will use Scikit-learn models, let's accelerate them first by using Intel Extension.

* vers. 20 - introduced selective rounding
* vers. 22 - adding sin/cos processing of day of the year at different time lags
* vers. 25 - updated festivities
* vers. 27 - added error analysis
* vers. 28 - revamped regression functional form
* vers. 30 - adopting AmbrosM feature engineering

In [ ]:
!pip install scikit-learn -U

In [ ]:
# Intel® Extension for Scikit-learn installation:
!pip install scikit-learn-intelex

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

##### We then load the data, it is necessary paying attention to convert the date into datetime

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import dateutil.easter as easter
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [ ]:
# Loading train and test data
train = pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv", parse_dates=['date'])
test = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv", parse_dates=['date'])

In [ ]:
train.dtypes

##### Before starting with EDA, it is important to check about the data structure. Apparently we have a combination of time series based on countries, stores and products. Let's first check if all the combinations appear in train and test.

In [ ]:
# figuring out the theoretically possible level combination
time_series = ['country', 'store', 'product']
combinations = 1
for feat in time_series:
    combinations *= train[feat].nunique()
    
print(f"There are {combinations} possible combinations")

In [ ]:
time_series = ['country', 'store', 'product']
country_store_product_train = train[time_series].drop_duplicates().sort_values(time_series)
country_store_product_test =test[time_series].drop_duplicates().sort_values(time_series)

cond_1 = len(country_store_product_train) == combinations
print(f"Are all theoretical combinations present in train: {cond_1}")
cond_2 = (country_store_product_train == country_store_product_test).all().all()
print(f"Are combinations the same in train and test: {cond_2}")

##### As a second step let's visualize how time is split between train and test.

In [ ]:
train_dates = train.date.drop_duplicates().sort_values()
test_dates = test.date.drop_duplicates().sort_values()

fig, ax = plt.subplots(1, 1, figsize = (11, 7))
cmap_cv = plt.cm.coolwarm

color_index = np.array([1] * len(train_dates) + [0] * len(test_dates))

ax.scatter(range(len(train_dates)), [.5] * len(train_dates),
           c=color_index[:len(train_dates)], marker='_', lw=15, cmap=cmap_cv,
           label='train', vmin=-.2, vmax=1.2)

ax.scatter(range(len(train_dates), len(train_dates) + len(test_dates)), [.55] * len(test_dates),
           c=color_index[len(train_dates):], marker='_', lw=15, cmap=cmap_cv,
           label='test', vmin=-.2, vmax=1.2)

tick_locations = np.cumsum([0, 365, 366, 365, 365, 365])
for i in (tick_locations):
    ax.vlines(i, 0, 2,linestyles='dotted', colors = 'grey')
    
ax.set_xticks(tick_locations)
ax.set_xticklabels([2015, 2016, 2017, 2018, 2019, 2020], rotation = 0)
ax.set_yticklabels(labels=[])
plt.ylim([0.45, 0.60])
ax.legend(loc="upper left", title="data")

plt.show()

##### Having four complete years available allows various types of testing and modelling. In this EDA we will limit to use the last year available (2018) as an hold-out, setting our baseline model to be able to forecast an entire year in the future.

##### As a last check we verify that no date is missing from train and test:

In [ ]:
missing_train = pd.date_range(start=train_dates.min(), end=train_dates.max()).difference(train_dates)
missing_test = pd.date_range(start=test_dates.min(), end=test_dates.max()).difference(test_dates)
print(f"missing dates in train: {len(missing_train)} and in test: {len(missing_test)}")

##### Having completed the checks, we process the datetime information, extracting it informative elements at different time granularities:

In [ ]:
# We create different time granularity

def process_time(df):
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['week'] = df['date'].dt.isocalendar().week
    df['week'][df['week']>52] = 52
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['dayofyear'] = df['date'].dt.dayofyear
    return df

train = process_time(train)
test = process_time(test)

##### We are ready to explore the data. In order to highlight the time series characteristics, we create panels of products x countries x shops. We start by aggregating at a year level.

In [ ]:
for product in ['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker']:
    print(f"\n--- {product} ---\n")
    fig = plt.figure(figsize=(20, 10), dpi=100)
    fig.subplots_adjust(hspace=0.25)
    for i, store in enumerate(['KaggleMart', 'KaggleRama']):
        for j, country in enumerate(['Finland', 'Norway', 'Sweden']):
            ax = fig.add_subplot(2, 3, (i*3+j+1))
            selection = (train['country']==country)&(train['store']==store)&(train['product']==product)
            selected = train[selection]
            selected.set_index('date').groupby('year')['num_sold'].mean().plot(ax=ax)
            ax.set_title(f"{country}:{store}")
    plt.show()

##### The first series of panels points out that the country effect is kind of indipendent from store and product. There is an underlying country dynamic that replicates the same no matter the shop or the product sold by it. We also notice that shops differentiate only for the level of sales.

##### Our next panels will explore seaasonality based on months:

In [ ]:
for product in ['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker']:
    fig = plt.figure(figsize=(20, 10), dpi=100)
    fig.subplots_adjust(hspace=0.25)
    for i, store in enumerate(['KaggleMart', 'KaggleRama']):
        for j, country in enumerate(['Finland', 'Norway', 'Sweden']):
            ax = fig.add_subplot(2, 3, (i*3+j+1))
            selection = (train['country']==country)&(train['store']==store)&(train['product']==product)
            selected = train[selection]
            for year in [2015, 2016, 2017, 2018]:
                selected[selected.year==year].set_index('date').groupby('month')['num_sold'].mean().plot(ax=ax, label=year)
            ax.set_title(f"{product} | {country}:{store}")
            ax.legend()
    plt.show()

##### Here we notice two important elements: seanality curves are different for each product and they also differ from year to year. Averaging the curves probably is safe bet for the future, as well as considering more relevant the recent years (thus weighting more the year 2018 for instance). For the sticker product, year 2017 seems particularly different from others.

##### We now proceed to examine seasonality even more in detail at a week level:

In [ ]:
for product in ['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker']:
    print(f"\n--- {product} ---\n")
    fig = plt.figure(figsize=(20, 10), dpi=100)
    fig.subplots_adjust(hspace=0.25)
    for i, store in enumerate(['KaggleMart', 'KaggleRama']):
        for j, country in enumerate(['Finland', 'Norway', 'Sweden']):
            ax = fig.add_subplot(2, 3, (i*3+j+1))
            selection = (train['country']==country)&(train['store']==store)&(train['product']==product)
            selected = train[selection]
            for year in [2015, 2016, 2017, 2018]:
                selected[selected.year==year].set_index('date').groupby('week')['num_sold'].mean().plot(ax=ax, label=year)
            ax.set_title(f"{country}:{store}")
            ax.legend()
    plt.show()

##### At a week level we see that differences are due to peaks. Peaks seem different in Spring. Probably is is Easter effect.

##### We now start obeserving recurrences at a monthly level:

In [ ]:
for product in ['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker']:
    print(f"\n--- {product} ---\n")
    fig = plt.figure(figsize=(20, 10), dpi=100)
    fig.subplots_adjust(hspace=0.25)
    for i, store in enumerate(['KaggleMart', 'KaggleRama']):
        for j, country in enumerate(['Finland', 'Norway', 'Sweden']):
            ax = fig.add_subplot(2, 3, (i*3+j+1))
            selection = (train['country']==country)&(train['store']==store)&(train['product']==product)
            selected = train[selection]
            for year in [2015, 2016, 2017, 2018]:
                selected[selected.year==year].set_index('date').groupby('day')['num_sold'].mean().plot(ax=ax, label=year)
            ax.set_title(f"{country}:{store}")
            ax.legend()
    plt.show()

##### The middle of the month usually presents less sales. The peak at the end may be influenced by seasonal peaks (end of year).

##### And we completed by inspecting at a day of the week level:

In [ ]:
for product in ['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker']:
    print(f"\n--- {product} ---\n")
    fig = plt.figure(figsize=(20, 10), dpi=100)
    fig.subplots_adjust(hspace=0.25)
    for i, store in enumerate(['KaggleMart', 'KaggleRama']):
        for j, country in enumerate(['Finland', 'Norway', 'Sweden']):
            ax = fig.add_subplot(2, 3, (i*3+j+1))
            selection = (train['country']==country)&(train['store']==store)&(train['product']==product)
            selected = train[selection]
            for year in [2015, 2016, 2017, 2018]:
                selected[selected.year==year].set_index('date').groupby('dayofweek')['num_sold'].sum().plot(ax=ax, label=year)
            ax.set_title(f"{country}:{store}")
            ax.legend()
    plt.show()

##### Friday and the week-end are the best days, but Sundays are not always at the same level as Saturdays (it depends on the year - why?).

##### Based on the information we got we now proceed to feature engineering and to enrich the data (using festivities and GDP data).

In [ ]:
festivities = pd.read_csv("../input/festivities-in-finland-norway-sweden-tsp-0122/nordic_holidays.csv",
                          parse_dates=['date'],
                          usecols=['date', 'country', 'holiday'])

In [ ]:
gdp = pd.read_csv("../input/gdp-20152019-finland-norway-and-sweden/GDP_data_2015_to_2019_Finland_Norway_Sweden.csv")
gdp = np.concatenate([gdp[['year', 'GDP_Finland']].values, 
                      gdp[['year', 'GDP_Norway']].values, 
                      gdp[['year', 'GDP_Sweden']].values])
gdp = pd.DataFrame(gdp, columns=['year', 'gdp'])
gdp['country'] = ['Finland']*5 + ['Norway']*5 +['Sweden']*5

##### We now process the data and scale it. Since EDA revealed how the different characteristics of the series are mostly main effects (country and store), we focus on finding the way to model the interaction between products and time.

In [ ]:
def process_target(df):
    target = pd.DataFrame({'row_id':df['row_id'], 'num_sold':df['num_sold']})
    target = target.sort_values('row_id').set_index('row_id')
    return target

def process_data(df):
    """Return a new dataframe with the engineered features"""

    new_df = pd.DataFrame({'gdp': np.log(df.merge(gdp, on=['country', 'year'], how='left')['gdp'].values),
                           'wd4': df.date.dt.weekday == 4, # Friday
                           'wd56': df.date.dt.weekday >= 5, # Saturday and Sunday
                          })

    # One-hot encoding (no need to encode the last categories)
    for country in ['Finland', 'Norway']:
        new_df[country] = df.country == country
    new_df['KaggleRama'] = df.store == 'KaggleRama'
    for product in ['Kaggle Mug', 'Kaggle Hat']:
        new_df[product] = df['product'] == product
        
    # Seasonal variations (Fourier series)
    # The three products have different seasonal patterns
    dayofyear = df.date.dt.dayofyear
    for k in range(1, 3):
        new_df[f'sin{k}'] = np.sin(dayofyear / 365 * 2 * np.pi * k)
        new_df[f'cos{k}'] = np.cos(dayofyear / 365 * 2 * np.pi * k)
        new_df[f'mug_sin{k}'] = new_df[f'sin{k}'] * new_df['Kaggle Mug']
        new_df[f'mug_cos{k}'] = new_df[f'cos{k}'] * new_df['Kaggle Mug']
        new_df[f'hat_sin{k}'] = new_df[f'sin{k}'] * new_df['Kaggle Hat']
        new_df[f'hat_cos{k}'] = new_df[f'cos{k}'] * new_df['Kaggle Hat']

    # End of year
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d)
                                      for d in range(24, 32)}),
                        pd.DataFrame({f"n-dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d) &
                                      (df.country == 'Norway')
                                      for d in range(24, 32)}),
                        pd.DataFrame({f"f-jan{d}":
                                      (df.date.dt.month == 1) & (df.date.dt.day == d) & 
                                      (df.country == 'Finland')
                                      for d in range(1, 14)}),
                        pd.DataFrame({f"jan{d}":
                                      (df.date.dt.month == 1) & (df.date.dt.day == d) &
                                      (df.country == 'Norway')
                                      for d in range(1, 10)}),
                        pd.DataFrame({f"s-jan{d}":
                                      (df.date.dt.month == 1) & (df.date.dt.day == d) & 
                                      (df.country == 'Sweden')
                                      for d in range(1, 15)})],
                       axis=1)
    
    # May
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"may{d}":
                                      (df.date.dt.month == 5) & (df.date.dt.day == d) 
                                      for d in list(range(1, 10))}),
                        pd.DataFrame({f"may{d}":
                                      (df.date.dt.month == 5) & (df.date.dt.day == d) & 
                                      (df.country == 'Norway')
                                      for d in list(range(18, 28))})],
                       axis=1)
    
    # June and July
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"june{d}":
                                      (df.date.dt.month == 6) & (df.date.dt.day == d) & 
                                      (df.country == 'Sweden')
                                      for d in list(range(8, 14))}),
                       ],
                       axis=1)
    
    # Last Wednesday of June
    wed_june_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-06-24')),
                                         2016: pd.Timestamp(('2016-06-29')),
                                         2017: pd.Timestamp(('2017-06-28')),
                                         2018: pd.Timestamp(('2018-06-27')),
                                         2019: pd.Timestamp(('2019-06-26'))})
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"wed_june{d}": 
                                      (df.date - wed_june_date == np.timedelta64(d, "D")) & 
                                      (df.country != 'Norway')
                                      for d in list(range(-4, 6))})],
                       axis=1)
    
    # First Sunday of November
    sun_nov_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-11-1')),
                                         2016: pd.Timestamp(('2016-11-6')),
                                         2017: pd.Timestamp(('2017-11-5')),
                                         2018: pd.Timestamp(('2018-11-4')),
                                         2019: pd.Timestamp(('2019-11-3'))})
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"sun_nov{d}": 
                                      (df.date - sun_nov_date == np.timedelta64(d, "D")) &
                                      (df.country != 'Norway')
                                      for d in list(range(0, 9))})],
                       axis=1)
    
    # First half of December (Independence Day of Finland, 6th of December)
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d) &
                                      (df.country == 'Finland')
                                      for d in list(range(6, 14))})],
                       axis=1)

    # Easter
    easter_date = df.date.apply(lambda date: pd.Timestamp(easter.easter(date.year)))
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"easter{d}": 
                                      (df.date - easter_date == np.timedelta64(d, "D"))
                                      for d in list(range(-2, 11)) + list(range(40, 48)) +
                                      list(range(50, 59))})],
                       axis=1)
    
    return new_df.astype(np.float32)


train_test = process_data(train.append(test))

processed_train = train_test.iloc[:len(train)].copy()
processed_test = train_test.iloc[len(train):].copy()

target = np.ravel(process_target(train))

##### Since EDA demonstrated that there is some kind of yearly change, we overweight more recent observations.

In [ ]:
def weighting(df, weights):
    return df.year.replace(weights).values
    
weights = weighting(train, {2015:1, 2016:1, 2017:1, 2018:1})

In [ ]:
processed_train.shape, train.shape, processed_test.shape, test.shape

##### We prepare all the evaluation measures, both at an aggregate level, with exp transformation and at an individual cases level (for error analysis)

In [ ]:
def SMAPE(y_true, y_pred):
    # From https://www.kaggle.com/cpmpml/smape-weirdness
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

def SMAPE_exp(y_true, y_pred):
    y_true = np.exp(y_true)
    y_pred = np.exp(y_pred)
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

def SMAPE_err(y_true, y_pred):
    # From https://www.kaggle.com/cpmpml/smape-weirdness
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return diff

##### We try modelling both an additive and a multiplicative predictor (for multiplicative you just need a log transoformation of the target because exponentiation of the sum of logs of the terms correspond to multiplications of the terms)

In [ ]:
# due to float calculations the computation is approximated
a = 5
b = 7
print(a * b) # pure multiplicative
print(np.exp(np.log(a) + np.log(b))) # multiplicative made additive by log

In [ ]:
from sklearn.linear_model import LinearRegression, HuberRegressor

reg = LinearRegression()
addictive_model = reg.fit(processed_train, target)
addictive_fit = addictive_model.predict(processed_train)
print(f"addictive fit: {SMAPE(y_true=target, y_pred=addictive_fit): 0.3f}")

In [ ]:
multiplicative_model = reg.fit(processed_train, np.log(target))
multiplicative_fit = np.exp(multiplicative_model.predict(processed_train))
print(f"multiplicative fit: {SMAPE(y_true=target, y_pred=multiplicative_fit):0.3f}")

##### It seems that the multiplicative model is better. We now use it to predict on the test set.

In [ ]:
def selective_rounding(preds, lower=0.3, upper=0.7):
    # selective rounding
    dec = preds % 1
    to_round = (dec<=lower)|(dec>=upper)
    preds[to_round] = np.round(preds[to_round])
    return preds

In [ ]:
reg = LinearRegression()

model = reg.fit(processed_train, np.log(target))
submission = pd.read_csv("../input/tabular-playground-series-jan-2022/sample_submission.csv")
preds = np.exp(model.predict(processed_test))

# rounding
preds = selective_rounding(preds, lower=0.3, upper=0.7)

submission.num_sold = preds
submission.to_csv("submission.csv", index=False)

##### For testing purposes we refit the model only on years 2015-2017 using 2018 as an hold-out test.

In [ ]:
log_target = np.log(target)
train_set = list(train.index[train.date<'2018-01-01'])
val_set = list(train.index[train.date>='2018-01-01'])
model = LinearRegression()

model.fit(processed_train.iloc[train_set], log_target[train_set])
preds = np.exp(model.predict(processed_train.iloc[val_set]))

# rounding
preds = selective_rounding(preds, lower=0.3, upper=0.7)

smape = SMAPE(y_true=target[val_set], y_pred=preds)
print(f"hold-out smape: {smape}")

##### As a closing work, we check the fit of the model (residual analysis) in order to figure out if we correctly specified the functional form of the predictor. Holidays are marked in the chart.

In [ ]:
import calendar

all_preds = np.exp(model.predict(processed_train))

year = 2017
for month in range(1, 13):
    print(f"\n*** {calendar.month_name[month]} ***\n")
    for product in ['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker']:
        print(f"\n--- {product} ---\n")
        fig = plt.figure(figsize=(20, 10), dpi=100)
        fig.subplots_adjust(hspace=0.25)
        for i, store in enumerate(['KaggleMart', 'KaggleRama']):
            for j, country in enumerate(['Finland', 'Norway', 'Sweden']):
                ax = fig.add_subplot(2, 3, (i*3+j+1))
                selection = train.row_id.isin(train.row_id[train.year==year])
                selected = train[selection].copy()
                selected['preds'] = all_preds[selection]
                selection = (selected['country']==country)&(selected['store']==store)&(selected['product']==product)
                selection = selection & (selected.month==month)
                selected = selected[selection].copy()
                grouped = selected.set_index('date').groupby('day')
                num_sold_grouped = grouped['num_sold'].mean()
                preds_grouped = grouped['preds'].mean()
                num_sold_grouped.plot(ax=ax, label='truth')
                preds_grouped.plot(ax=ax, label='preds')
                fests = festivities[(festivities.country==country)&(festivities.date.dt.month==month)&(festivities.date.dt.year==year)]
                fest_idx = (fests.date.dt.day).values
                if len(fest_idx) > 0:
                    plt.plot(fest_idx, num_sold_grouped.values[fest_idx-1], 'o', color='red')
                ax.set_title(f"{calendar.month_name[month]}|{country}:{store}")
                ax.legend()
        plt.show()

### If you liked the notebook, consider to upvote, thank you and happy Kaggling!